## FHV October 2019 

In [27]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

24/02/25 12:07:03 WARN Utils: Your hostname, Poojas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.43.130 instead (on interface en0)
24/02/25 12:07:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 12:07:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/25 12:07:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [13]:
spark.version

'3.5.0'

In [12]:
!wc -l fhv_tripdata_2019-10.csv

 1897494 fhv_tripdata_2019-10.csv


In [15]:
df_fhv = spark.read.option("header", "true").csv('fhv_tripdata_2019-10.csv')

In [17]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [19]:
df_fhv.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [20]:
# Changing Datetime Columns to Timestamp Type , LocationID Columns to Integer Type
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [21]:
df_fhv = spark.read.option("header", "true").schema(schema).csv('fhv_tripdata_2019-10.csv')

In [22]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [23]:
df_fhv.repartition(6)
df_fhv.write.parquet('data/fhv/')

24/02/25 12:26:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [24]:
!ls -lhR data/fhv/

total 64512
-rw-r--r--  1 poojasingh  staff     0B Feb 25 12:26 _SUCCESS
-rw-r--r--  1 poojasingh  staff   3.4M Feb 25 12:26 part-00000-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.5M Feb 25 12:26 part-00001-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.4M Feb 25 12:26 part-00002-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.5M Feb 25 12:26 part-00003-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.4M Feb 25 12:26 part-00004-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.4M Feb 25 12:26 part-00005-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   3.4M Feb 25 12:26 part-00006-9a54847b-f0f1-4526-ad28-64a05709845d-c000.snappy.parquet
-rw-r--r--  1 poojasingh  staff   2.5M Feb 25 12:26 part-00007-9a54847b-f0f1-4526-ad28-

## Records on 15th October 2019

In [30]:
df_fhv.withColumn('pickup_date', F.to_date(df_fhv.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

## Longest Trip in Hours

In [31]:
df_fhv.registerTempTable('fhv_2019_10')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [39]:
df_fhv.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [40]:
df_fhv.withColumn('duration', (F.col('dropOff_datetime').cast('long') - F.col('pickup_datetime').cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df_fhv.pickup_datetime)) \
    .groupBy('pickup_date').max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-----------------+
|pickup_date|    max(duration)|
+-----------+-----------------+
| 2019-10-11|         631152.5|
| 2019-10-28|         631152.5|
| 2019-10-31|87672.44083333333|
| 2019-10-01|70128.02805555555|
| 2019-10-17|           8794.0|
+-----------+-----------------+



In [42]:
spark.sql("""
SELECT
    TO_DATE(pickup_datetime) AS pickup_date,
    CAST(MAX((UNIX_TIMESTAMP(dropoff_datetime) - UNIX_TIMESTAMP(pickup_datetime)) / 3600) AS BIGINT) AS duration_in_hrs
FROM fhv_2019_10
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
""").show()

+-----------+---------------+
|pickup_date|duration_in_hrs|
+-----------+---------------+
| 2019-10-11|         631152|
| 2019-10-28|         631152|
| 2019-10-31|          87672|
| 2019-10-01|          70128|
| 2019-10-17|           8794|
| 2019-10-26|           8784|
| 2019-10-30|           1464|
| 2019-10-25|           1056|
| 2019-10-02|            769|
| 2019-10-03|            745|
+-----------+---------------+



## Least Frequent Pickup Locations

In [5]:
# Loading Zone CSV file
!curl -O https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12322  100 12322    0     0   7346      0  0:00:01  0:00:01 --:--:--  73432586      0  0:00:04  0:00:01  0:00:03  2585


In [43]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [45]:
df_zones.write.parquet('data/zones', mode='overwrite')

In [46]:
!ls -lh data/zones/

total 16
-rw-r--r--  1 poojasingh  staff     0B Feb 25 12:50 _SUCCESS
-rw-r--r--  1 poojasingh  staff   5.8K Feb 25 12:50 part-00000-e0ca44aa-92a1-4cf2-954d-3004fd23dfd0-c000.snappy.parquet


In [47]:
df_zones.registerTempTable('zones')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [48]:
df_zones.registerTempTable('zones')


In [49]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [53]:
spark.sql("""
    SELECT
        pu.Zone AS pickup_location, COUNT(1)
    FROM fhv_2019_10 fhv 
    LEFT JOIN zones pu ON fhv.PULocationID = pu.LocationID
    LEFT JOIN zones do ON fhv.DOLocationID = do.LocationID
    GROUP BY 1
    ORDER BY 2 ASC
    LIMIT 20;
""").show()

+--------------------+--------+
|     pickup_location|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+

